In [ ]:
# Google Harry Potter image collection

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import requests
from selenium.common.exceptions import TimeoutException

# 设置 WebDriver 路径
edge_driver_path = r"C:\Users\WDMX\Downloads\edgedriver_win64\msedgedriver.exe"
edge_service = Service(executable_path=edge_driver_path)

# 启用无界面模式，并添加 User-Agent
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
options.page_load_strategy = 'eager'  # 页面加载策略

driver = webdriver.Edge(service=edge_service, options=options)

def download_images_from_google(query, num_images, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    search_url = f"https://www.google.com/search?q={query}&tbm=isch"
    driver.get(search_url)
    time.sleep(10)  # 增加等待时间

    # 滚动页面加载更多图片
    for _ in range(20):  # 增加滚动次数
        driver.execute_script("window.scrollBy(0, 3000);")
        time.sleep(3)

    # 模拟点击“加载更多”按钮
    try:
        load_more_button = driver.find_element(By.XPATH, '//input[@value="Show more results"]')
        if load_more_button:
            load_more_button.click()
            time.sleep(5)  # 等待更多图片加载
    except Exception as e:
        print("No 'Show more results' button found or error:", e)

    # 捕获 TimeoutException 异常
    try:
        WebDriverWait(driver, 40).until(EC.presence_of_all_elements_located((By.XPATH, '//img[contains(@class,"rg_i")]')))
    except TimeoutException:
        print("Timed out waiting for images to load.")
        driver.save_screenshot(f"debug_screenshot_{query}.png")  # 截图调试

    # 查找图片元素
    img_elements = driver.find_elements(By.XPATH, '//img[contains(@class,"rg_i")]')
    print(f"Found {len(img_elements)} images on the page.")  # 检查图片数量

    downloaded = 0

    for i, img_elem in enumerate(img_elements):
        img_url = img_elem.get_attribute("src") or img_elem.get_attribute("data-src") or img_elem.get_attribute("srcset")
        print(f"Image {i}: {img_url}")  # 打印每个图片的 URL
        if img_url and downloaded < num_images:
            try:
                img_data = requests.get(img_url).content
                img_path = os.path.join(save_dir, f"{query}_{i}.jpg")
                with open(img_path, 'wb') as handler:
                    handler.write(img_data)
                downloaded += 1
            except Exception as e:
                print(f"Failed to download image {i}: {e}")
    
    print(f"Downloaded {downloaded} images for {query}.")

# 调用函数
characters = ["Harry Potter", "Hermione Granger", "Ron Weasley"]
for character in characters:
    download_images_from_google(character, 200, f"C:/Users/WDMX/Desktop/harry_potter_images/{character}")

driver.quit()
